# 🎭 ROOP - Face Swap on Google Colab
This notebook provides a simple interface to run ROOP face swapping in Google Colab with GPU acceleration.

## 📦 Step 1: Setup Environment
Run this cell once to install all dependencies and CUDA libraries.

In [ ]:
# Clone ROOP repository
!git clone https://github.com/s0md3v/roop.git --depth 1
%cd roop

# Install CUDA libraries
!apt-get update
!apt-get install -y libcuda11-0 libcublas11 libcufft10 libcudnn8 libnccl2 cuda-runtime-11-2 ffmpeg

# Install Python dependencies
!pip install -q -r requirements.txt

print("✅ Setup completed!")

## 🔍 Step 2: Check GPU Availability
Verify that your GPU and execution providers are properly configured.

In [ ]:
import onnxruntime as ort
import torch

print("🔍 GPU Status:")
print(f"   PyTorch CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("\n🎯 ONNX Runtime Providers:")
providers = ort.get_available_providers()
for i, provider in enumerate(providers, 1):
    print(f"   {i}. {provider}")

print("\n✅ Ready to process videos!")

## 📤 Step 3: Upload Files
Upload your source image and target video.

In [ ]:
from google.colab import files
import os

# Create directories
os.makedirs('source', exist_ok=True)
os.makedirs('input_videos', exist_ok=True)
os.makedirs('output_videos', exist_ok=True)

print("📤 Upload source image (JPG/PNG):")
source_files = files.upload()
if source_files:
    for filename in source_files:
        os.rename(filename, f'source/{filename}')
        print(f"   ✓ Saved: source/{filename}")

print("\n📤 Upload target video (MP4):")
video_files = files.upload()
if video_files:
    for filename in video_files:
        os.rename(filename, f'input_videos/{filename}')
        print(f"   ✓ Saved: input_videos/{filename}")

## 🎬 Step 4: Process Video
Configure and run the face swap processing.

In [ ]:
import subprocess
import os
from pathlib import Path

# Get uploaded files
source_files = list(Path('source').glob('*'))
video_files = list(Path('input_videos').glob('*'))

if not source_files:
    print("❌ No source image found!")
elif not video_files:
    print("❌ No target video found!")
else:
    source_path = str(source_files[0])
    video_path = str(video_files[0])
    output_path = f'output_videos/{video_files[0].stem}_result.mp4'
    
    print(f"📸 Source: {source_path}")
    print(f"🎬 Video: {video_path}")
    print(f"💾 Output: {output_path}")
    print("\n🚀 Starting processing...\n")
    
    cmd = [
        'python', 'run.py',
        '-s', source_path,
        '-t', video_path,
        '-o', output_path,
        '--frame-processor', 'face_swapper', 'face_enhancer',
        '--execution-provider', 'cuda',
        '--keep-fps',
        '--many-faces',
        '--max-memory', '10',
        '--execution-threads', '8',
        '--output-video-encoder', 'h264_nvenc',
        '--output-video-quality', '18',
        '--temp-frame-format', 'png'
    ]
    
    result = subprocess.run(cmd, capture_output=False)
    
    if result.returncode == 0:
        print(f"\n✅ Processing completed!")
        print(f"   Output: {output_path}")
    else:
        print(f"\n❌ Processing failed with code {result.returncode}")

## 📥 Step 5: Download Results
Download your processed video.

In [ ]:
from google.colab import files
from pathlib import Path

output_files = list(Path('output_videos').glob('*.mp4'))

if not output_files:
    print("❌ No output videos found")
else:
    print("📥 Downloading results...")
    for output_file in output_files:
        print(f"   ⬇️  {output_file.name}")
        files.download(str(output_file))